## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-17-18-53-53 +0000,nypost,Major Northeastern cities expected to get up t...,https://nypost.com/2026/01/17/us-news/northeas...
1,2026-01-17-18-50-52 +0000,bbc,US justice department investigating Minnesota ...,https://www.bbc.com/news/articles/czx1wgvpr1zo...
2,2026-01-17-18-50-44 +0000,nyt,Dueling Protests Begin in Minneapolis as City ...,https://www.nytimes.com/2026/01/17/us/minneapo...
3,2026-01-17-18-48-03 +0000,nypost,"Boy, 9, sues 12-year-old after bloody daycare ...",https://nypost.com/2026/01/17/world-news/9-yea...
4,2026-01-17-18-46-10 +0000,nypost,Trailer park at the end of the world: ‘Mutiny ...,https://nypost.com/2026/01/17/world-news/trail...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2418/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,54
16,ice,21
61,iran,15
63,new,15
20,minneapolis,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2026-01-17-17-42-00 +0000,wsj,President Trump said he would impose 10% tarif...,https://www.wsj.com/politics/policy/trump-to-h...,117
232,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...,114
243,2026-01-16-20-23-34 +0000,nypost,Trump admin slams LA Mayor Karen Bass after sh...,https://nypost.com/2026/01/16/us-news/trump-ad...,108
58,2026-01-17-13-33-32 +0000,nypost,Iran’s supreme leader blasts Trump as a ‘crimi...,https://nypost.com/2026/01/17/world-news/ayato...,106
129,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,117,2026-01-17-17-42-00 +0000,wsj,President Trump said he would impose 10% tarif...,https://www.wsj.com/politics/policy/trump-to-h...
157,78,2026-01-17-00-59-04 +0000,cbc,U.S. Justice Department probing Minnesota Gov....,https://www.cbc.ca/news/world/tim-walz-jacob-f...
45,58,2026-01-17-14-43-35 +0000,bbc,Iran supreme leader admits thousands killed du...,https://www.bbc.com/news/articles/ckglee733wno...
33,49,2026-01-17-16-49-16 +0000,nypost,Ex-Nickelodeon child star Kianna Underwood’s d...,https://nypost.com/2026/01/17/us-news/ex-nicke...
232,44,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...
104,37,2026-01-17-10-00-29 +0000,wapo,Thousands prayed for a new Venezuela. One moth...,https://www.washingtonpost.com/world/interacti...
18,36,2026-01-17-17-50-20 +0000,bbc,Blair and Rubio among names on senior executiv...,https://www.bbc.com/news/articles/c07xv92vrz2o...
220,34,2026-01-16-21-24-32 +0000,cbc,Here's what you need to know about the Canada-...,https://www.cbc.ca/news/politics/what-is-in-ca...
240,32,2026-01-16-20-34-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
139,32,2026-01-17-02-34-27 +0000,startribune,Federal judge bars ICE agents from ‘retaliatin...,https://www.startribune.com/judge-blocks-ice-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
